In [16]:
import torch
from torch import nn,optim,Tensor
from torchvision import transforms
from torch.utils.data import DataLoader, random_split,Dataset
from tqdm import tqdm

In [3]:
# sys.path.append(os.path.abspath(".."))
from models import DoubleConv,VGG13
from models.DataLoader import FaceDataset

In [10]:
# Training Variable

SEED = 42
ROOT_PATH = "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning/data/raw/Images"
IN_CHANNELS = 3
OUT_CHANNELS = 3
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 20

# Check is nvidia GPU is available

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=(67.3686, 52.9727, 44.3268),std=(42.2854, 35.1020, 32.3488)),
    transforms.Resize((224,224),antialias=True)
])

# Initializing Datasets
dataset = FaceDataset(ROOT_PATH,transform)
generator = torch.Generator().manual_seed(SEED)

train_data,attack_data, test_data = random_split(dataset,[0.7,0.1,0.2],generator=generator)

print('Total number of dataset:',len(dataset))
print('Train                  :',len(train_data))
print('Attack                 :', len(attack_data))
print('Test                   :', len(test_data))


train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True, drop_last=True, generator=generator)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, generator=generator)


# Setting up the Model

model = VGG13(IN_CHANNELS,OUT_CHANNELS).to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

Total number of dataset: 5000
Train                  : 3500
Attack                 : 500
Test                   : 1000


In [11]:
def checkpoint( model, filename: str )->None:
    
    folder_path = "home/ychien13/my_github/CSE467_Privacy/Image_Poisoning/src/checkpoints"
    
    return torch.save(model.state_dict(), os.path.join(folder_path,filename))

In [ ]:
# img, label = next(iter(train_loader)) 

for epoch in range(EPOCHS):
    
    model.train()
    correct=0
    total=0
    
    for idx, (img, label) in tqdm(enumerate(train_loader),total=len(train_loader)):
        
        img = img.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs,label)
        # print(loss)
        loss.backward()
        optimizer.step()
    
        correct+=(outputs.argmax(dim=1) == label).sum().item()
        total+=label.size(0)

    
    print(f'Epoch [{epoch+1}/{EPOCHS}] | {correct}/{total} | Accuracy: {correct/total:.2f}')

    checkpoint(model,)